### When you run this program:
* A series of fetch requests will be made to airnda.co to fetch property listings
* These property listings will then be merged with an Airbnb dataset obtained online
* When this program completes, check the file saved in airbnb_dfw_property_listings variable for all property listings
* This program takes about 10 minutes to run

#### NOTES:
* There is an access_token used to access airdna.co listings. To obtain the token, load an airdna page e.g https://www.airdna.co/vacation-rental-data/app/us/texas/dallas/overview, check XHR traffic in the network tab of browser developer tools and copy the access_token. Paste the value in the access_token variable and record the last updated date. See next point for tip on obtaining access_token.
* Tip: Load chromedriver window from airdna_metrics_scraper but do not close window by commenting out the line driver.close(). Only run it for a single market.
* If you get a KeyError: 'properties' - update access_token (see above)
* city_id_dict was compiled by observing airdna traffic - the index id is an Airdna internal market id for each city

In [11]:
import json
import csv
import requests
import os
import pandas as pd
import plotly.express as px
# import time
import datetime
from datetime import date
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

prop_lists_path = 'data/prop_lists'
prop_lists_json_path = prop_lists_path + '/json/'
prop_lists_csv_path = prop_lists_path + '/csv/'
access_token = 'MjkxMTI|8b0178bf0e564cbf96fc75b8518a5375' # last updated 11/14/20
start_year = '2019' # Prop list start year
datasets_path = 'data/datasets'
airbnb_texas_rental_dataset= datasets_path + '/Airbnb_texas_rental_data.csv'
airdna_dfw_property_list = prop_lists_csv_path + '/airdna_dfw_property_list.csv'
airbnb_dfw_property_listings = 'data/airbnb_dfw_property_listings.csv'
datestamp = date.today().strftime("%Y%m%d")

city_id_dict = {
    '79752' : 'dallas',
    '79888' : 'fort-worth',
    '80413' : 'plano',
    '80045' : 'irving',
    '79491' : 'arlington',
    '79901' : 'frisco',
    '79949' : 'grand-prairie',
    '80237' : 'mckinney',
    '79771' : 'denton',
    '80482' : 'richardson'
}

markets = {
    'dallas' : 'Dallas',
    'fort-worth' : 'Fort-Worth',
    'plano' : 'Plano',
    'irving' : 'Irving',
    'arlington' : 'Arlington',
    'frisco' : 'Frisco',
    'grand-prairie' : 'Grand Prairie',
    'mckinney' : 'McKinney',
    'denton' : 'Denton',
    'richardson' : 'Richardson'
}

def setUp():
    """Initiate folders used for read/write operations"""
    if not os.path.exists(prop_lists_json_path):
        os.makedirs(prop_lists_json_path)
    if not os.path.exists(prop_lists_csv_path):
        os.makedirs(prop_lists_csv_path)

def fetch_prop_listings():
    """Fetch Airdna property listings"""
    print('Fetching Airdna property listings...')
    begin_time = datetime.datetime.now()
    setUp()
    
    for city_id, city_name in city_id_dict.items():
        url='https://api.airdna.co/v1/market/property_list?access_token={}&city_id={}&start_month=1&start_start_year={}&number_of_months=12&currency=native&show_regions=true'.format(
            access_token, city_id, start_year
        )
        prop_list_json = requests.get(url)
        data = prop_list_json.content
        file_path = '{}{}_property_list_{}.json'.format(prop_lists_json_path, city_name, datestamp)
        with open(file_path, 'wb') as f:
            f.write(data)

    convert_prop_listing_json_data_to_csv()
    merge_property_csv_lists()
    merge_airbnb_datasets_with_airdna_property_lists()
    completion_time = datetime.datetime.now() - begin_time
    print('Completed in {} minutes!'.format(completion_time))

def convert_prop_listing_json_data_to_csv():
    """Converts json property listings to csv format"""
    print('Converting json property lists to csv...')
    unwanted_prop_keys = ['occ', 'revenue', 'regions', 'id', 'platforms', 'homeaway_property_id', 'm_homeaway_property_id']
    for city_id, city_name in city_id_dict.items():
        file_path = '{}{}_property_list_{}.json'.format(prop_lists_json_path, city_name, datestamp)
        with open(file_path) as f:
            data = json.load(f)
        property_data = data['properties']
        # Clean data and add additional calculated fields
        for prop in property_data:
            # Additional columns
            prop['airbnb_url'] = 'https://www.airbnb.com/rooms/' + str(prop['airbnb_property_id'])
            # Check & record if Airbnb listing is active
            prop['active_listing'] = 0
#             if (requests.head(prop['airbnb_url']).status_code == 200):
            if (fetch_http_status_code(prop['airbnb_url']) == 200):
                prop['active_listing'] = 1
            prop['market'] = markets[city_name]
            # Filter out unwanted property keys
            for unwanted_prop_key in unwanted_prop_keys:
                prop.pop(unwanted_prop_key)

        # Write data to csv
        csv_file_path = '{}{}_property_list_{}.csv'.format(prop_lists_csv_path, city_name, datestamp)
        data_file = open(csv_file_path, 'w', encoding='utf-8', newline='')
        csv_writer = csv.writer(data_file)
        count = 0

        for prop in property_data: 
            if count == 0: 
                # Write headers
                header = prop.keys() 
                csv_writer.writerow(header) 
                count += 1
            # Write data 
            csv_writer.writerow(prop.values()) 

        data_file.close()

def merge_property_csv_lists():
    """Merge all csv property lists into a single file"""
    print('Merging csv property lists...')
#     prop_lists_csv_path = 'prop_lists/csv/'
    df = pd.DataFrame()
    for city_id, city_name in city_id_dict.items():
        file_path = '{}{}_property_list_{}.csv'.format(prop_lists_csv_path, city_name, datestamp)
        df = df.append(pd.read_csv(file_path))
    df.to_csv(airdna_dfw_property_list, index=False)

def merge_airbnb_datasets_with_airdna_property_lists():
    """Merge Airdna property list and Airbnb dataset into a single file"""
    print('Merging Airbnb datasets with Airdna property lists...')
    df1 = pd.read_csv(airbnb_texas_rental_dataset)
    df1 = df1[
        df1.city.str.contains('Dallas', case=False)
        | df1.city.str.contains('Fort Worth', case=False)
        | df1.city.str.contains('Plano', case=False)
        | df1.city.str.contains('Irving', case=False)
        | df1.city.str.contains('Arlington', case=False)
        | df1.city.str.contains('Frisco', case=False)
        | df1.city.str.contains('Grand Prairie', case=False)
        | df1.city.str.contains('McKinney', case=False)
        | df1.city.str.contains('Denton', case=False)
        | df1.city.str.contains('Richardson', case=False)
    ]
    df1 = df1[df1.city != 'Lake Dallas']
    df1['average_daily_rate'] = df1['average_daily_rate'].str.replace('$', '')
    df1['city'] = df1['city'].str.replace('Fort Worth', 'Fort-Worth', case=False)
    df1['bedrooms'] = df1['bedrooms'].str.replace('Studio', '0')
#     df1.head()
    df2 = pd.read_csv(airdna_dfw_property_list)
    df2 = df2.rename(columns={'market': 'city', 'adr': 'average_daily_rate', 'airbnb_url' : 'url'})
    df2['average_daily_rate'] = round(df2['average_daily_rate'])
#     df2.head()
    df3 = pd.concat([df2, df1], ignore_index=True)
    df3.to_csv(airbnb_dfw_property_listings, index = False)

def fetch_http_status_code(url):
    """Fetch http status code"""
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(url)
    return response.status_code

fetch_prop_listings()

Fetching Airdna property listings...
Converting json property lists to csv...
Merging csv property lists...
Merging Airbnb datasets with Airdna property lists...
Completed in 0:09:49.556019 minutes!
